In [ ]:
# @todo description

In [ ]:
import cv2
from enum import Enum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

In [ ]:
# Params
imgfile = '/media/docker/workspace/xraybones/res/xray.jpg'
imgfile = '/media/docker/workspace/xraybones/res/xray2.png'

In [ ]:
# Functions
def convolve2D(image: np.array([]), kernel: np.array([]), padding=0, strides=1) -> np.array([]):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output

In [ ]:
# XRay Hand class
class ImageType(Enum):
    NONE, RAW = -1, 1
    
class BodyPartType(Enum):
    NONE, HAND = -1, 0

class BoneDetector():
    
    figsize = 15
    bodyparttype = BodyPartType.NONE
    image = np.array([])
    images = pd.DataFrame(columns=['img', 'imagetype', 'description'])
    
    def __init__(self, imagefile: str) -> None:
        """
        Init X-Ray Hand detector
            :param imagefile
        """
        # Read image
        self.image = cv2.imread(imagefile)
        if self.image is None:
            raise Exception("Could not read the image.")
        
        self.image = cv2.cvtColor(src=self.image, code=cv2.COLOR_BGR2GRAY)
        
        self.images = self.images.append({'img': self.image, 'imagetype': ImageType.RAW, 'description': 'raw image'}, ignore_index=True)
            
    def show_all_images(self) -> None:
        """
        Show all images
            :param imagetype
        """
        fig = plt.figure(figsize=(figsize,figsize))
        
        for img in self.images.iterrows():
            plt.imshow(img.img, cmap='gray')
    
    def show_image(self, imagetype: ImageType) -> None:
        """
        Show specified image
            :param imagetype
        """
        fig = plt.figure(figsize=(self.figsize,self.figsize))
        
        for index, row in self.images.iterrows():
            if row.imagetype == imagetype:
                plt.imshow(row.img, cmap='gray')
    
    def show_image_overlay(self, imagetype: ImageType) -> None:
        """
        Overlay image with imagetype over raw image
            :param imagetype
        """
        fig = plt.figure(figsize=(self.figsize,self.figsize))
        
        image_raw_rgb = cv2.cvtColor(src=self.image, code=cv2.COLOR_GRAY2RGB)
        
        for index, row in self.images.iterrows():
            if row.imagetype == imagetype:
                image_raw_r = cv2.cvtColor(src=row.img, code=cv2.COLOR_GRAY2RGB)
                image_raw_r[:, :, 1] = 0
                image_raw_r[:, :, 2] = 0
        
        #
        vis = cv2.addWeighted(image_raw_r,1.0,image_raw_rgb,1.0,0)
        
        plt.imshow(vis)


In [ ]:
# Init bone detector
bonedetector = BoneDetector(imgfile)
bonedetector.show_image(ImageType.RAW)
#bonedetector.show_image_overlay(ImageType.RAW)

## Use k-means to remove background color intensity

In [ ]:
# k-means color clustering
#Read the original image grayscale color
img = cv2.imread(imgfile, 0) 

# Get image height and width
rows, cols = img.shape[:]

# Image two-dimensional pixel conversion to one-dimensional
data = img.reshape((rows * cols))
data = np.float32(data)

# Definition Center (type,max_iter,epsilon)
criteria = (cv2.TERM_CRITERIA_EPS +
            cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS

# K-Means clustering into 4 categories
compactness, labels, centers = cv2.kmeans(data, 2, None, criteria, 10, flags)

# Generate final image
res = centers[labels.flatten()]
dst = res.reshape((img.shape[0],img.shape[1]))

print('First cluster intensity %f' % min(centers))

# Display image
#fig = plt.figure(figsize=(figsize,figsize))
#plt.imshow(dst, 'gray')  

In [ ]:
# Crop image
imgraw = cv.imread(imgfile)

cropping = 0.08
imgraw = imgraw[ int(cropping*imgraw.shape[0]):int((1.-cropping)*imgraw.shape[0]), 
          int(cropping*imgraw.shape[1]):int((1.-cropping)*imgraw.shape[1])]
imgraw = cv2.cvtColor(src=imgraw, code=cv2.COLOR_BGR2GRAY)

## Hand and Finger Detection

In [ ]:
# Extract convex hull and defects
# https://medium.com/analytics-vidhya/hand-detection-and-finger-counting-using-opencv-python-5b594704eb08
# https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html

import cv2 as cv
import numpy as np

fig = plt.figure(figsize=(15, 15))

# Loading exposure images into a list
img = imgraw.copy()

# Blur image
img_blur = cv2.GaussianBlur(img,(15,15),0)

# Threshold (binary image)
low_thres = int(min(centers))
ret,img_thres = cv2.threshold(img_blur, low_thres, 255, cv2.THRESH_BINARY)
img_eq = cv2.equalizeHist(img_thres)

plt.subplot(1,3,1); plt.xticks([]),plt.yticks([]); plt.imshow(img_eq, cmap='gray')

# Morphological (remove outliers)
kernel = np.ones((15, 15), 'uint8')
img_morph = cv2.morphologyEx(img_eq, cv2.MORPH_CLOSE, kernel)
#equalize = cv2.morphologyEx(equalize, cv2.MORPH_CLOSE, kernel)

plt.subplot(1,3,2); plt.xticks([]),plt.yticks([]); plt.imshow(img_morph, cmap='gray')

# Contours
contours, hierarchy = cv2.findContours(img_morph, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = max(contours, key=lambda x: cv2.contourArea(x))

# Convex hull
hull = cv2.convexHull(contours)

# Hull defects
hull = cv2.convexHull(contours, returnPoints=False)
defects = cv2.convexityDefects(contours, hull)

img2 = imgraw.copy()
cv2.drawContours(img2, [contours], -1, (255, 255, 255), 2)

plt.subplot(1,3,3); plt.xticks([]),plt.yticks([]); plt.imshow(img2, cmap='gray')

# Extract hand
#value = 240
#mask = img > value
#img_new = np.where((255 - img) < value, 255, img+value)
#plt.imshow(img_new, cmap='gray')

#kernel = np.ones((10, 10), 'uint8')
#dilate_img = cv2.dilate(img_new, kernel, iterations=3)
#erosion_img = cv2.erode(dilate_img, kernel, iterations=3)
#opening = cv2.morphologyEx(img_new, cv2.MORPH_OPEN, kernel)
#closing = cv2.morphologyEx(img_new, cv2.MORPH_CLOSE, kernel)
#gradient = cv2.morphologyEx(closing, cv2.MORPH_GRADIENT, kernel)


#img_list = [cv.imread(fn) for fn in img_fn]
#exposure_time = np.array(15.0, dtype=np.float32)
#merge_debevec = cv.createMergeDebevec()

In [ ]:
img2 = imgraw.copy()

if defects is not None:
    defcnt = 0

fingergaps = np.array([])

for i in range(defects.shape[0]):  # calculate the angle
    s, e, f, d = defects[i][0]
    start = tuple(contours[s][0])
    end = tuple(contours[e][0])
    far = tuple(contours[f][0])
    a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
    b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
    c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  #      cosine theorem
    if angle <= np.pi / 2:  # angle less than 90 degree, treat as fingers
        defcnt += 1
        cv.circle(img2, far, 10, [255, 0, 0], -1)
        fingergaps = np.append(fingergaps, [far[0], -far[1]])
        
fingergaps = fingergaps.flatten().reshape(-1,2)

if defcnt > 0:
    defcnt = defcnt+1
    
# Visualize
fig = plt.figure(figsize=(15, 15))

cv.putText(img2, '%d Fingers detected' % defcnt, (0, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2, cv.LINE_AA)
cv2.drawContours(img2, [contours], -1, (255, 255, 255), 2)

In [ ]:
# Visualize convex hull and defects
img2 = imgraw.copy()

cnt = contours

defects_arr = np.array([])

for i in range(defects.shape[0]):
    s,e,f,d = defects[i,0]
    start = tuple(cnt[s][0])
    end = tuple(cnt[e][0])
    far = tuple(cnt[f][0])
    
    if d > 500:
        continue
    elif far[1] > 0.9*img2.shape[0]: # ignore points at bottom
        continue
    defects_arr = np.append(defects_arr, [far[0], -far[1]])
    #cv.circle(img2,far,10,[255,0,0],-1)

# K-Means Clustering of fingertip defects
defects_arr = defects_arr.reshape(-1,2)
kmeans = KMeans(n_clusters=5, random_state=0).fit(defects_arr)
kmeans.labels_
fingertips = kmeans.cluster_centers_

In [ ]:
# Visualize
fig = plt.figure(figsize=(15, 15))

cv.putText(img2, '%d Fingers detected' % defcnt, (0, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2, cv.LINE_AA)
cv2.drawContours(img2, [contours], -1, (255, 255, 255), 2)

fig = plt.figure(figsize=(15, 15))
plt.imshow(img2, cmap='gray')

for fingergap in fingergaps:
    plt.scatter(fingergap[0], -fingergap[1], c='r', marker='.', s=200)
    
for fingertip in fingertips:
    plt.scatter(fingertip[0], -fingertip[1], c='r', marker='x', s=200)

## Apply HDR
https://docs.opencv.org/3.4/d2/df0/tutorial_py_hdr.html

In [ ]:
# Apply HDR
# 
import cv2 as cv
import numpy as np

# Loading exposure images into a list
img = cv2.imread(imgfile)
value=-50
img2 = np.where((255 - img) < value, 255, img+value)
value=100
img3 = np.where((255 - img) < value, 255, img+value)

img_list = [img, img2, img3]
exposure_times = np.array([10., 15., 20.], dtype=np.float32)

# Merge exposures to HDR image
merge_debevec = cv2.createMergeDebevec()
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())

## Detect Shapes
https://www.pyimagesearch.com/2016/02/08/opencv-shape-detection/

## K-Means Clustering
https://realpython.com/k-means-clustering-python/
https://scikit-learn.org/stable/modules/clustering.html#clustering
https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-auto-examples-cluster-plot-dbscan-py

In [ ]:
# k-Means histogram clusterin
img = imgraw.copy()

# Get image height and width
rows, cols = img.shape[:]

# Image two-dimensional pixel conversion to one-dimensional
data = img.reshape((rows * cols))
data = np.float32(data)

# Definition Center (type,max_iter,epsilon)
criteria = (cv2.TERM_CRITERIA_EPS +
            cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS

# K-Means clustering into 4 categories
compactness, labels, centers = cv2.kmeans(data, 2, None, criteria, 10, flags)

In [ ]:
print(centers)
hist,bins = np.histogram(img.ravel(),256,[0,256])

#plt.hist(image.ravel(),256,[0,256]); 
plt.plot(bins[1:],hist,[0,256])
plt.scatter(centers, np.zeros(len(centers)), c='r')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = imgraw.copy()

ret,image = ret,image = cv2.threshold(image, int(centers[1]), 255, cv2.THRESH_BINARY)

#equalize= cv2.equalizeHist(thresh_image)

#canny_image = cv2.Canny(equalize,250,255)
#canny_image = cv2.convertScaleAbs(canny_image)
#kernel = np.ones((3,3), np.uint8)
#dilated_image = cv2.dilate(canny_image,kernel,iterations=1)

#contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#contours= sorted(contours, key = cv2.contourArea, reverse = True)[:10]
#c=contours[0]

#final = cv2.drawContours(img, [c], -1, (255,0, 0), 3)
#mask = np.zeros(image.shape,np.uint8)

#new_image = cv2.drawContours(mask,[c],0,255,-1,)
#new_image = cv2.bitwise_and(image, image, mask=equalize)

#cv2.namedWindow("new",cv2.WINDOW_NORMAL)
#cv2.imshow("new",new_image)
#cv2.waitKey(0)

# Visualize data

fig = plt.figure(figsize=(figsize,figsize))
plt.imshow(image, cmap='gray')

## Image Filtering
https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html

## Convolution
https://medium.com/analytics-vidhya/2d-convolution-using-python-numpy-43442ff5f381

In [ ]:
# k-means color clustering
#Read the original image grayscale color
img = imgraw.copy()

#Get image height and width
rows, cols = img.shape[:]

#Image two-dimensional pixel conversion to one-dimensional
data = img.reshape((rows * cols))
data = np.float32(data)

#Definition Center (type,max_iter,epsilon)
criteria = (cv2.TERM_CRITERIA_EPS +
            cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS

#K-Means clustering into 4 categories
compactness, labels, centers = cv2.kmeans(data, 2, None, criteria, 10, flags)

#Generate final image
res = centers[labels.flatten()]
dst = res.reshape((img.shape[0],img.shape[1]))

#Display image
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(figsize,figsize))

plt.imshow(dst, 'gray')  
plt.show()

In [ ]:
# Get image coordinates as points with threshold
plt.figure(figsize=(figsize, figsize))
image = imgraw.copy()
indices = np.where(image > 180)
plt.axis('equal')
plt.scatter(indices[1], -indices[0], marker='.', c='k', s=1)

In [ ]:
# Display mutliple images
titles = [u'The original image', u'Cluster image']  
images = [img, dst]  
for i in range(2):  
    plt.subplot(1,2,i+1), plt.imshow(images[i], 'gray'), 
    plt.title(titles[i])  
    plt.xticks([]),plt.yticks([])  

## Image Operations

Links:
https://heartbeat.fritz.ai/opencv-python-cheat-sheet-from-importing-images-to-face-detection-52919da36433
https://www.kdnuggets.com/2019/08/introduction-image-segmentation-k-means-clustering.html
https://docs.opencv.org/3.4/d8/dbc/tutorial_histogram_calculation.html

In [ ]:
# Read and Convert color
image = cv2.imread('/media/docker/artifacts/imgs/xray.jpg') 
image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY)

In [ ]:
if False:
    # Crop
    cropped = image[10:500, 500:2000]
    # Convert Color
    #image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY) 
    # Blur
    image = cv2.blur(image, ksize=(30, 30))
    # Flip
    image = cv2.flip(image, flipCode=1)
    # Resize
    scale=0.5
    image = cv2.resize(image, (int(image.shape[1]*scale), int(image.shape[0]*scale)) , interpolation = cv2.INTER_AREA)
    # Canny Edge
    image = cv2.Canny(image,150,200)
    # Morphological (dilate, erode, open, close, gradient)
    image = cv2.dilate(image, kernel, iterations=3)
    image = cv2.erode(image, kernel, iterations=3)
    image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
    # Threshold
    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html
    ret,image = cv2.threshold(img, 10, 255, cv2.THRESH_BINARY)
    
# Histogram
hist,bins = np.histogram(image.ravel(),256,[0,256])
plt.plot(bins[1:],hist,[0,256])


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(figsize,figsize))
plt.imshow(image, cmap='gray')